In [ ]:
# ## pipeline of the model


# data transformation
# data Model
# best model selection
# store it into artifacts

# prediction pipeline will be created

# transformation
# model predict - clear_output


# app development

# CICD pipeline
# deployment

# git actions



How things will work in piepleine
1. we will have data loading
2. we will calll funtions that will help us with 

In [4]:
# ======================================================
# data_cleaning.py
# ======================================================
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from Classifier.utils.common import load_data





# ------------------------------------------------------
# 2. DEFINE FEATURE GROUPS (based on your EDA)
# ------------------------------------------------------
categorical_features = [
    'Percentile (GTET 100 Max Voltage)',
    'Percentile (GTET 200 Max Voltage)',
    'Percentile (CAISO)',
    'Install Type',
    'Urban or Rural',
    'Region'
]

continuous_features = [
    'Acres',
    'Distance to Substation (Miles) GTET 100 Max Voltage',
    'Distance to Substation (Miles) GTET 200 Max Voltage',
    'Distance to Substation (Miles) CAISO',
    'Shape__Area',
    'Shape__Length'
]


target_col = 'Solar Technoeconomic Intersection'



ModuleNotFoundError: No module named 'Classifier'

In [ ]:

# ------------------------------------------------------
# 3. BUILD TRANSFORMERS
# ------------------------------------------------------

# 3.1 Log transform only for "Shape__Length"
def log_transformer(df):
    """Apply log1p to Shape__Length."""
    df = df.copy()
    df['Shape__Length'] = np.log1p(df['Shape__Length'])
    return df

log_tf = FunctionTransformer(log_transformer, validate=False)

# 3.2 Imputer (handles any missing values)
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# 3.3 Scaler
scaler = StandardScaler()

# 3.4 Encoder for categorical variables
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# ------------------------------------------------------
# 4. CREATE COLUMN TRANSFORMER
# ------------------------------------------------------

# Numerical pipeline (impute → log transform → scale)
numeric_pipeline = Pipeline(steps=[
    ('imputer', num_imputer),
    ('log_transform', log_tf),
    ('scaler', scaler)
])

# Categorical pipeline (impute → one-hot)
categorical_pipeline = Pipeline(steps=[
    ('imputer', cat_imputer),
    ('encoder', encoder)
])

# Combine both
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, continuous_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)


# ------------------------------------------------------
# 5. FINAL PIPELINE (ready for model training)
# ------------------------------------------------------
def build_cleaning_pipeline():
    """Returns the preprocessing pipeline."""
    return preprocessor


# ------------------------------------------------------
# 6. APPLY PIPELINE + SAVE SCALER
# ------------------------------------------------------
def preprocess_and_save(df, scaler_path='scaler.pkl'):
    """Clean dataset, save scaler, and return transformed arrays."""
    X = df[categorical_features + continuous_features]
    y = df[target_col]

    pipe = build_cleaning_pipeline()
    X_clean = pipe.fit_transform(X)

    # Save the fitted scaler (from numeric pipeline)
    fitted_scaler = pipe.named_transformers_['num'].named_steps['scaler']
    with open(scaler_path, 'wb') as f:
        pickle.dump(fitted_scaler, f)
        print(f"💾 Scaler saved to {scaler_path}")

    print(f"Data cleaned and transformed. Final shape: {X_clean.shape}")
    return X_clean, y, pipe


# ------------------------------------------------------
# 7. Example usage
# ------------------------------------------------------
if __name__ == "__main__":
    df = load_data('data/your_dataset.csv')

    X_clean, y, pipe = preprocess_and_save(df, scaler_path='artifacts/scaler.pkl')

    # Optionally split for modeling
    X_train, X_test, y_train, y_test = train_test_split(X_clean, y, test_size=0.2, random_state=42)
    print("✅ Train/test split complete:", X_train.shape, X_test.shape)
